# Imports

In [2]:
import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-289aa57c-2278-4474-b9c2-634d6db862fa.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4226_Show_an_example_of_trades_data_to_OHLCV_conversion'
  hash='bbaaf4f45'
  # Last commits:
    * bbaaf4f45 vlady    CmTask4226: notebook with the examples of ohlcv resampling        (    8 days ago) Tue Apr 18 18:48:02 2023  (HEAD -> CmTask4226_Show_an_example_of_trades_data_to_OHLCV_conversion, origin/CmTask4226_Show_an_example_of_trades_data_to_OHLCV_conversion)
    * b805d5864 Grigorii Pomazkin Cm task4240 tweak 

# Get couple hours worth of trades data

In [79]:
import im_v2.ccxt.data.extract.extractor as imvcdexex

exchange_id = "kraken"
contract_type = "spot"
ccxt_extractor = imvcdexex.CcxtExtractor(exchange_id, contract_type)

data_type = "trades"
currency_pair = "BTC/USDT"
start_timestamp = pd.Timestamp("2023-04-26T10:00:00+00:00")
end_timestamp = pd.Timestamp("2023-04-26T15:00:00+00:00")
data = ccxt_extractor.download_data(
    data_type,
    exchange_id,
    currency_pair,
    start_timestamp=start_timestamp,
    end_timestamp=end_timestamp,
    depth=10,
)

In [80]:
data.head()

,timestamp,symbol,side,price,amount,end_download_timestamp
0,1682503217983,BTC/USDT,buy,28883.3,0.015878,2023-04-26 18:37:56.480359+00:00
1,1682503218952,BTC/USDT,sell,28883.2,0.173111,2023-04-26 18:37:56.480359+00:00
2,1682503221325,BTC/USDT,buy,28883.3,0.000913,2023-04-26 18:37:56.480359+00:00
3,1682503231466,BTC/USDT,sell,28883.2,0.173198,2023-04-26 18:37:56.480359+00:00
4,1682503231547,BTC/USDT,sell,28883.2,0.250000,2023-04-26 18:37:56.480359+00:00


In [81]:
data = data.set_index(data.timestamp.apply(pd.Timestamp, unit="ms"), drop=False)

In [82]:
df_ohlc = data.price.resample("1Min").ohlc()

In [83]:
df_ohlc.head()

,open,high,low,close
timestamp,,,,
2023-04-26 10:00:00,28883.3,28883.3,28848.7,28848.7
2023-04-26 10:01:00,28868.8,28871.5,28868.8,28871.5
2023-04-26 10:02:00,28852.9,28887.8,28849.5,28886.5
2023-04-26 10:03:00,28862.8,28862.8,28862.8,28862.8
2023-04-26 10:04:00,NaN,NaN,NaN,NaN


In [84]:
df_volume = data.amount.resample("1Min").sum()

In [85]:
resample_data = pd.concat(
    [df_ohlc, df_volume],
    axis=1,
)

In [86]:
resample_data.rename(columns={"amount": "volume"}, inplace=True)

In [87]:
result = resample_data.copy()
result["open"] = resample_data["open"].fillna(
    resample_data["close"].ffill(), limit=1
)
result["close"] = resample_data["close"].fillna(
    resample_data["open"].bfill(), limit=1
)

In [88]:
result["open"] = result["open"].fillna(result["close"].ffill())
result["close"] = result["close"].fillna(result["close"].ffill())

In [89]:
result["high"] = result["high"].fillna(result[["open", "close"]].max(axis=1))
result["low"] = result["low"].fillna((result[["open", "close"]].min(axis=1)))

# get some recent OHLCV data from kraken

In [95]:
data_type = "ohlcv"
currency_pair = "BTC/USDT"
start_timestamp = pd.Timestamp("2023-04-26T10:00:00+00:00")
end_timestamp = pd.Timestamp("2023-04-26T15:00:00+00:00")
data = ccxt_extractor.download_data(
    data_type,
    exchange_id,
    currency_pair,
    start_timestamp=start_timestamp,
    end_timestamp=end_timestamp,
    depth=10,
)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.00it/s]
/app/im_v2/ccxt/data/extract/extractor.py:210: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  hdbg.dassert(all_bars_df.timestamp.is_monotonic)


In [96]:
data["timestamp"].agg(["min", "max"]).apply(pd.to_datetime, unit="ms")

min   2023-04-26 10:00:00
max   2023-04-26 15:00:00
Name: timestamp, dtype: datetime64[ns]

# Compare resampled data and raw data

In [97]:
data.head()

,timestamp,open,high,low,close,volume,end_download_timestamp
0,1682503200000,28883.3,28883.3,28848.7,28848.7,1.231659,2023-04-26 18:40:03.529006+00:00
1,1682503260000,28868.8,28871.5,28868.8,28871.5,0.020595,2023-04-26 18:40:03.529006+00:00
2,1682503320000,28852.9,28887.8,28849.5,28886.5,0.699707,2023-04-26 18:40:03.529006+00:00
3,1682503380000,28862.8,28862.8,28862.8,28862.8,0.009384,2023-04-26 18:40:03.529006+00:00
4,1682503440000,28862.8,28862.8,28862.8,28862.8,0.000000,2023-04-26 18:40:03.529006+00:00


In [98]:
data = data.set_index(data.timestamp.apply(pd.Timestamp, unit="ms"), drop=True)

In [99]:
data = data.drop(columns=["timestamp", "end_download_timestamp"])

In [100]:
data.head(10)

,open,high,low,close,volume
timestamp,,,,,
2023-04-26 10:00:00,28883.3,28883.3,28848.7,28848.7,1.231659
2023-04-26 10:01:00,28868.8,28871.5,28868.8,28871.5,0.020595
2023-04-26 10:02:00,28852.9,28887.8,28849.5,28886.5,0.699707
2023-04-26 10:03:00,28862.8,28862.8,28862.8,28862.8,0.009384
2023-04-26 10:04:00,28862.8,28862.8,28862.8,28862.8,0.000000
2023-04-26 10:05:00,28836.7,28836.7,28836.7,28836.7,0.000340
2023-04-26 10:06:00,28836.7,28836.7,28836.7,28836.7,0.000000
2023-04-26 10:07:00,28824.7,28826.4,28824.7,28826.4,0.500100
2023-04-26 10:08:00,28827.5,28827.5,28827.5,28827.5,0.010181


In [101]:
result.head(10)

,open,high,low,close,volume
timestamp,,,,,
2023-04-26 10:00:00,28883.3,28883.3,28848.7,28848.7,1.231659
2023-04-26 10:01:00,28868.8,28871.5,28868.8,28871.5,0.020595
2023-04-26 10:02:00,28852.9,28887.8,28849.5,28886.5,0.699707
2023-04-26 10:03:00,28862.8,28862.8,28862.8,28862.8,0.009384
2023-04-26 10:04:00,28862.8,28862.8,28836.7,28836.7,0.000000
2023-04-26 10:05:00,28836.7,28836.7,28836.7,28836.7,0.000340
2023-04-26 10:06:00,28836.7,28836.7,28836.7,28836.7,0.000000
2023-04-26 10:07:00,28824.7,28826.4,28824.7,28826.4,0.500100
2023-04-26 10:08:00,28827.5,28827.5,28827.5,28827.5,0.010181


In [76]:
result.equals(data)

False

In [102]:
result.loc[[pd.Timestamp("2023-04-26 10:04:00")]]

,open,high,low,close,volume
timestamp,,,,,
2023-04-26 10:04:00,28862.8,28862.8,28836.7,28836.7,0.0


In [103]:
data.loc[[pd.Timestamp("2023-04-26 10:04:00")]]

,open,high,low,close,volume
timestamp,,,,,
2023-04-26 10:04:00,28862.8,28862.8,28862.8,28862.8,0.0


In [53]:
result.compare(data)

open              high               low             close             volume           
                        self    other     self    other     self    other     self    other      self      other
timestamp                                                                                                       
2023-04-26 10:04:00      NaN      NaN      NaN      NaN  28836.7  28862.8  28836.7  28862.8       NaN        NaN
2023-04-26 10:28:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.063772   0.063772
2023-04-26 10:41:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  2.927847   2.927847
2023-04-26 10:47:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.086549   0.086549
2023-04-26 10:48:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.006083   0.006083
2023-04-26 10:49:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.235648   0.235648
2023-04-26 10:50:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  3.428959   3.428959
2023-04-26 10:54:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.345053   0.345053
2023-04-26 11:06:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.005263   0.005263
2023-04-26 11:08:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.169598   0.169598
2023-04-26 11:15:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.038865   0.038865
2023-04-26 11:18:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.083236   0.083236
2023-04-26 11:24:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.014227   0.014227
2023-04-26 11:43:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.044154   0.044154
2023-04-26 12:07:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.477727   0.477727
2023-04-26 12:08:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.000623   0.000623
2023-04-26 12:17:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.532940   0.532940
2023-04-26 12:19:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.142250   0.142250
2023-04-26 12:22:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.448609   0.448609
2023-04-26 12:23:00      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  0.009732   0.009732
2023-04-26 12:29:00      NaN      NaN  29746.4  29803.7      NaN      NaN  29746.4  29730.3  5.181890   5.956821
2023-04-26 12:30:00  29746.4  29740.1  29746.4  29740.1  29746.4  29626.4  29746.4  29701.7  0.000000   0.326186
2023-04-26 12:31:00  29746.4  29694.2  29746.4  29694.2  29746.4  29678.1  29746.4  29678.1  0.000000   0.007622
2023-04-26 12:32:00  29746.4  29699.2  29746.4  29891.1  29746.4  29699.2  29746.4  29787.0  0.000000   3.463355
2023-04-26 12:33:00  29746.4  29787.3  29746.4  29984.6  29746.4  29787.3  29746.4  29923.1  0.000000   4.875838
2023-04-26 12:34:00  29746.4  29946.3  29746.4  29991.9  29746.4  29862.1  29746.4  29862.1  0.000000   6.757000
2023-04-26 12:35:00  29746.4  29862.5  29746.4  29913.3  29746.4  29862.5  29746.4  29883.5  0.000000   0.309692
2023-04-26 12:36:00  29746.4  29895.7  29746.4  29895.7  29746.4  29895.7  29746.4  29895.7  0.000000   0.017265
2023-04-26 12:37:00  29746.4  29872.2  29746.4  29872.2  29746.4  29838.3  29746.4  29850.9  0.000000   0.395849
2023-04-26 12:38:00  29746.4  29848.1  29746.4  29859.8  29746.4  29811.9  29746.4  29811.9  0.000000   0.021935
2023-04-26 12:39:00  29746.4  29815.3  29746.4  29835.4  29746.4  29814.0  29746.4  29831.6  0.000000   1.806833
2023-04-26 12:40:00  29746.4  29840.1  29746.4  29848.2  29746.4  29840.1  29746.4  29842.6  0.000000   0.007191
2023-04-26 12:41:00  29746.4  29835.1  29746.4  29848.9  29746.4  298

In [42]:
data.shape

(300, 5)

In [43]:
resample_data.shape

(300, 5)

In [38]:
resample_data.iloc[-10:]

,open,high,low,close,volume
timestamp,,,,,
2023-04-26 14:50:00,29647.5,29651.5,29647.5,29651.5,3.251480
2023-04-26 14:51:00,29662.7,29669.3,29662.7,29669.3,0.439908
2023-04-26 14:52:00,29702.3,29714.2,29686.2,29686.2,0.265983
2023-04-26 14:53:00,29688.2,29688.2,29677.8,29677.8,0.043044
2023-04-26 14:54:00,29697.3,29697.3,29686.7,29686.7,0.648219
2023-04-26 14:55:00,29674.3,29679.4,29672.9,29679.4,0.192516
2023-04-26 14:56:00,29688.2,29696.1,29688.2,29691.0,1.024820
2023-04-26 14:57:00,29713.5,29713.5,29685.9,29685.9,0.218909
2023-04-26 14:58:00,29704.3,29706.0,29704.3,29706.0,0.000862


In [41]:
data.iloc[-10:]

,open,high,low,close,volume
timestamp,,,,,
2023-04-26 14:50:00,29647.5,29651.5,29647.5,29651.5,3.251480
2023-04-26 14:51:00,29662.7,29669.3,29662.7,29669.3,0.439908
2023-04-26 14:52:00,29702.3,29714.2,29686.2,29686.2,0.265983
2023-04-26 14:53:00,29688.2,29688.2,29677.8,29677.8,0.043044
2023-04-26 14:54:00,29697.3,29697.3,29686.7,29686.7,0.648219
2023-04-26 14:55:00,29674.3,29679.4,29672.9,29679.4,0.192516
2023-04-26 14:56:00,29688.2,29696.1,29688.2,29691.0,1.024820
2023-04-26 14:57:00,29713.5,29713.5,29685.9,29685.9,0.218909
2023-04-26 14:58:00,29704.3,29706.0,29704.3,29706.0,0.000862


In [40]:
data.drop(data.tail(1).index, inplace=True)